## Проверяем размеры

In [ ]:
!du -h

In [ ]:
!du -Lh ./*

In [ ]:
!head ./trainGraph/part-04998-0a6ae864-587c-4028-b321-a43d239ffa2f-c000.csv

In [ ]:
!head ./coreDemography/part-r-00000

In [ ]:
!pip install psutil

__152 мегабайта - piece of cake!__

### Применительно к нашей задаче

* Первый прогноз можно построить по счетчику общих друзей
* Выданный граф не симетричный


## Попытка №1

Рекомендуем в качестве друзей тех, у кого много общих друзей с нами. Для этого втягиваем в память словарь вида [я->мои друзья], и его же в "развернутом виде" [я->те у кого я в друзьях]

### Готовим окружение

In [ ]:
import csv, gzip, os, random, numpy, math, timeit, copy

from collections import defaultdict

# Пути к данным
dataPath = "./"
graphPath = os.path.join(dataPath, "trainGraph")
demographyPath = os.path.join(dataPath, "coreDemography")
csv.field_size_limit(200000)

# Утилитка для замера памяти
def memory_usage():
    process = psutil.Process(os.getpid())
    print("Memory usage {0!s} mb.".format(process.memory_info()[0] / 1024 / 1024))

## Попытка №1 - читаем данные

In [ ]:
%%time
mineFriends = defaultdict(list)
friendsOfMine = defaultdict(list)

# Итерируемся по файлам в папке
for file in [f for f in os.listdir(graphPath) if f.endswith("csv")]:
    csv_reader = csv.reader(open(os.path.join(graphPath, file)), delimiter='\t')
    # А теперь по строкам в файле
    for line in csv_reader:
        user = int(line[0]) 
        # Разбираем идшки и маски друзей
        for friendship in line[1].replace("{(", "").replace(")}", "").split("),("):
            parts=friendship.split(",")
            friend = int(parts[0])
            mineFriends[user] += [friend]
            friendsOfMine[friend] += [user]

## Попытка №1 - тревожные сигналы

2 минуты на загрузку 152 мегабайт явно долговато, а что по памяти?

In [ ]:
memory_usage()

Все бы ничего, но два словарика в памяти __уже__ заняли больше трех гигабайт и дальше ворочать их будет неудобно. Но поробуем...

## Попытка №1 - пробуем расчет

In [ ]:
%%time
friendsOfFriends = defaultdict(lambda: defaultdict(int))

for (user,friends) in mineFriends.iteritems():
    for friend in friends:
        for theirFriend in friendsOfMine[friend]:
            friendsOfFriends[user][theirFriend] += 1

## Попытка №1 - работа над ошибками

In [ ]:
memory_usage()

### Применительно к нашей задаче

* Оценим объем данных
* Не забудем о накладных расходах!

## Разбор полетов - исследуем данные

Если с ходу не получилось, значит пришло время задуматся о том, как использовать память эффективнее. Для начала давайте попробуем прикинуть базовые показатели графа с которым работаем.

In [ ]:
numUsersFrom = 0
numLinks = 0
maxUserIdFrom = 0
maxUserIdTo = 0
allIds = set()

for file in [f for f in os.listdir(graphPath) if f.endswith("csv")]:
    csv_reader = csv.reader(open(os.path.join(graphPath, file)), delimiter='\t')
    for line in csv_reader:
        user = int(line[0])
        numUsersFrom += 1
        maxUserIdFrom = max(user,maxUserIdFrom)
        allIds.add(user)
        for friendship in line[1].replace("{(", "").replace(")}", "").split("),("):
            parts=friendship.split(",")
            friend = int(parts[0])
            allIds.add(friend)
            numLinks += 1
            maxUserIdTo = max(maxUserIdTo,friend)
            
print("Num users from {0}, max {1}, num total {2}, max {3}, num links {4}".format(
        numUsersFrom, maxUserIdFrom, len(allIds), maxUserIdTo,numLinks))

### Оценим объем нужной памяти

На одну связь надо 2 идишки, которые 4-х байтовый инт

In [ ]:
numLinks * (4 + 4) / 1024 / 1024

__Почему вместо 300Мб мы получили сразу 3Гб? Почему??!!__

## Разбор полетов - лезем в код

```c
#define PyDict_MINSIZE 8
typedef struct _dictobject PyDictObject;
struct _dictobject {
  ...
  PyDictEntry *ma_table;
  ...
};

typedef struct {
  Py_ssize_t me_hash;
  PyObject *me_key;
  PyObject *me_value;
} PyDictEntry;
```

## Хэш-таблица с открытой адресацией

<img src="https://upload.wikimedia.org/wikipedia/commons/b/bf/Hash_table_5_0_1_1_1_1_0_SP.svg" width="800" />

## Разбор полетов - считаем накладные расходы

* На запись
  * +8 байт на указатель на запись
  * +4 байта на хэш
  * +8*2 байтов на указатели на ключ/значение
* +30% свободных ячеек
* х2 свободных ячеек при копировании

In [ ]:
(numLinks * (8 + (8 + 4 + 16)) + 1.3 * 2 * 8 * numLinks) / 1024 / 1024

Становится яснее!

## А ведь есть еще вопрос локальности данных!

<center>
<img src="https://s3.amazonaws.com/media-p.slid.es/uploads/71982/images/2310246/figure04-caching-pyramid.png" width="600"/>
</center>

| Source   | Latency       |
| ---------|:-------------:|
| L1 Cache |  1-2 ns       |
| L2 Cache |  3-5 ns       |
| L3 Cache |  10-40 ns     |
| DRAM     |  60-100 ns    |



## Чем дальше в лес, тем толще партизаны - боксинг
```c
#define PyObject_HEAD                   \
    _PyObject_HEAD_EXTRA                \
    Py_ssize_t ob_refcnt;               \
    struct _typeobject *ob_type;

typedef struct {
    PyObject_HEAD
    long ob_ival;
} PyIntObject;
```

Кладя инт в "коробку" получаем 20 байт вместо 4-х, итого:

In [ ]:
(numLinks * ((20 + 20) + (8 + 4 + 16)) + 1.3 * 2 * 8 * numLinks) / 1024 / 1024

## Разбор полетов - все проблемы вскрыли?

__Проблема заполнености__ - общие друзья есть у гораздо большего числа пар пользователей.

In [ ]:
numUsersFrom * numUsersFrom * 4 / 1024 / 1024

### Применительно к нашей задаче

* Считаем общих друзей только там, где это нужно (входят в тестовое множество)

## Попытка №2

* Выбираем структуру данных
* Втягиваем правильно
* Считаем общих друзей для целевых юзеров
* Сабмитим
* Профит!

## Выбираем структуры данных

### Словари

* Легко пользоватся.
* Но _ОЧЕНЬ_ много накладных расходов: указатель на запись, указатель на ключ/значение, объектная обертка для ключа и значения.
* Итого: на пару инт-инт _накладных_ расходов 48 байт.
* Данные размазаны по памяти и фрагментированны - кеш процессора резко терят эффективность.

### Списки

* Если ключ инт - можно использовать вместо словаря.
* НО! Список в питоне это массив указателей, а значит накладные расходы на указатель и объектную обертку ключа. остаются - 20 байт накладных расходов на запись :(.
* Проблема с потерей эффективности кеша сохраняется.

### Массивы numpy

* На пару инт-инт нужно всего 4 байта.
* Кеш процессора работает.
* Но нужно чтобы ключи шли последовательно (к счастью, в наших даных это так почти так: максимальный ид 16619131 при общем числе 12417564).
* Для того чтобы заполнить массив _очень_ желательно знать его размер изначально.
* Если делать двумерный массив 100к на 16м - будет очень, очень, очень больно...

### Спарс матрицы scipy

* Предназначены для хранения двухмерных матриц с очень большим количеством нулей.
* Эффективные варианты хранят в виде нескольких массивов numpy.
* COO - три массива одинаковой длинны моделируют списов троек [(i,j,v)]. Просто строить, легко интегрировать с пандой.
* CSR - один маленьки массив с индексом начала для каждого ряда (монотонно растет), и два массива равно длинны с ид колонки и значением: [{i : [(j,v)]}]. Более компактна в памяти, быстрые опреации.
* CSC - тоже самое что CSR, но индексированны колонки.

#### CSR matrix
<img src="http://hamukazu.com/wp-content/uploads/2014/12/csr_matrix.png" width="800"/>

#### CSC matrix
<img src="http://hamukazu.com/wp-content/uploads/2014/12/csc_matrix2.png" width="800"/>

## Попытка № 2 - наш выбор

* Инициализруем граф в виде COO матрицы, затем переводим в CSR.
* Развернутый граф получаем с помощью транспонирования.
* Общих друзей считаем умножая матрицы.

#### Умножение матриц? WTF?

<img src="https://www.mathsisfun.com/algebra/images/matrix-multiply-a.gif" width="800"/>

## Попытка №2 - готовим окружение

In [ ]:
# Почистим шлак и оставим константы чтоб потом не пресчитывать
del allIds
maxUserId = 16619131
numLinks = 38031656

In [ ]:
# Эффективные массивы простых типов
import numpy
# Работа с матрицами (подсчет общих друзей реализован как умножение матрицы графа самое на себя)
import scipy
from scipy.sparse import coo_matrix, csr_matrix

## Попытка №2 - читаем матрицу

In [ ]:
formUser = numpy.zeros( (numLinks), dtype=numpy.int32 ) 
toUser = numpy.zeros( (numLinks), dtype=numpy.int32 ) 
data = numpy.ones( (numLinks), dtype=numpy.int16 ) 

current = 0

# Итерируемся по файлам в папке... 
for file in [f for f in os.listdir(graphPath) if f.endswith("csv")]:
    csv_reader = csv.reader(open(os.path.join(graphPath, file)), delimiter='\t')

    for line in csv_reader:
        user = int(line[0]) 
        maxUserId = max(user,maxUserId)
        # Разбираем идшки и маски друзей
        for friendship in line[1].replace("{(", "").replace(")}", "").split("),("):
            parts=friendship.split(",")
            # Записываем связь в массивы и двигаем указатель
            formUser[current] = user
            friend = int(parts[0])
            toUser[current] = friend
            maxUserId = max(friend,maxUserId)
            current += 1
            

Собственно из массивов создаем COO матрицу

In [ ]:
fullMatrix = coo_matrix(
    (data, (formUser, toUser)),
    shape=(maxUserId + 1, maxUserId + 1)).tocsr()

В матрице три массива, соответственно легко померять сколько памяти она занимает

In [ ]:
(fullMatrix.data.nbytes + fullMatrix.indptr.nbytes + fullMatrix.indices.nbytes) / (1024 * 1024)

Теперь аккуратно вычистим ненужное

In [ ]:
del formUser
del toUser
del data  

## Попытка №2 - считаем общих друзей для пользователей, по которым необходимо сформировать предсказание

In [ ]:
validationUsers = set()
csv_reader = csv.reader(open(os.path.join(dataPath, "prediction.csv")))
for line in csv_reader: validationUsers.add(int(line[0]))

In [ ]:
validationMatrix = copy.deepcopy(fullMatrix)

for i in range(maxUserId + 1):
    if i not in validationUsers:
        ptr = validationMatrix.indptr[i]
        ptr_next = validationMatrix.indptr[i+1]
        if ptr != ptr_next:
            validationMatrix.data[ptr:ptr_next].fill(0)
            

In [ ]:
validationMatrix.eliminate_zeros()

In [ ]:
%%time
commonFriends = validationMatrix.dot(fullMatrix)

In [ ]:
memory_usage()

In [ ]:
len(validationUsers)

## Что будет, если мы попробуем посчитать общих друзей для всех пользователей

In [ ]:
%%time
commonFriends = fullMatrix.dot(fullMatrix)

In [ ]:
memory_usage()

## Семинар

Для каждого пользователя посчитать оценку его возраста, как средние от возрастов его друзей

In [ ]:
demographyPath

In [ ]:
birthDates = numpy.zeros(maxUserId + 1, dtype=numpy.int32)

# Iterate all files in demography path
for file in [f for f in os.listdir(demographyPath) if f.endswith("gz")]:
    csvinput = gzip.open(os.path.join(demographyPath, file))
    csv_reader = csv.reader(csvinput, delimiter='\t')
   
    # Extract birth date from each line
    for line in csv_reader:
        user = int(line[0])
        birthDates[user] = int(line[2]) if line[2] != '' else 0

In [ ]:
print ("Memory used by demography: %s (mb)" % (birthDates.nbytes / (1024 * 1024)))